# Exercise 3: Interatomic potentials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
import sympy as sp

# a) Interatomic potential shape

<span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> Sketch a typical interatomic potential (energy as a function of interatomic distance $r$) and explain your choices. What are the key contributions to this potential? In particular, what is the behavior as $r \to 0$ and $r \to +\infty$?

<div class="alert alert-block alert-light">

## Solution

A typical interatomic potential incorporates two key physical contributions:

1. Short-range repulsion (dominant as $r \to 0$):
   At very small interatomic distances, the potential energy becomes strongly positive due to the overlap of electron clouds, which is prohibited by the Pauli exclusion principle. This repulsive interaction is commonly modeled in different forms across potential functions. For example:

   * In the Lennard-Jones potential the repulsion is represented by a steeply rising $\left(1/r\right)^{12}$ term.
   * In the Morse potential, the repulsion is captured by the exponential term $e^{-2a(r - r_e)}$, where $a$ is a measure of bond stiffness and $r_e$ is the equilibrium bond distance.
   * More sophisticated potentials, such as the Born-Mayer or Buckingham potentials, also use exponential or modified inverse power forms for short-range repulsion.

2. Long-range attraction (dominant as $r \to \infty$):
   At larger distances, the potential is dominated by attractive van der Waals (dispersion) interactions, which arise from correlated fluctuations of instantaneous dipoles. These are typically modeled by terms that decay as $-1/r^6$. For example:

   * The Lennard-Jones potential includes a $-\left(1/r\right)^6$ term to represent dispersion forces.
   * The Morse potential includes an attractive term of the form $-2e^{-a(r - r_e)}$, which smoothly approaches zero as $r \to \infty$.
   * In more complex models like the London dispersion potential, the attraction can also include higher-order multipole contributions depending on the system.

These two competing effects (strong short-range repulsion and weaker long-range attraction) together create a potential well with a minimum at an intermediate equilibrium distance, corresponding to the most stable atomic separation.

---

## The Stillinger-Weber (SW) potential for Si

The **Stillinger–Weber (SW) potential** for silicon was the first practical cluster potential to reproduce the **diamond cubic structure** as the most stable configuration. It captures not only the correct equilibrium bond lengths and angles but also accurately reflects the **energetic cost of distorting** the tetrahedral geometry, making it a foundational model for covalent systems.

Originally designed for **silicon**, the Stillinger–Weber potential uses the form:

#### Two-body term:
$$
\phi_2(r) = A \varepsilon \left[ B \left( \frac{\sigma}{r} \right)^p - \left( \frac{\sigma}{r} \right)^q \right] \exp\left( \frac{\sigma}{r - a\sigma} \right)
$$

**The exponential function is added to introduce a cutoff radius ($r_{\mathrm{cut}} = a \sigma$) while mantaining the function smooth and continuous.**

#### Three-body term:
$$
\phi_3(r_{ij}, r_{ik}, \theta) = \lambda \varepsilon \left[ \cos \theta - \cos \theta_0 \right]^2 \exp\left( \frac{\gamma \sigma}{r_{ij} - a\sigma} \right) \exp\left( \frac{\gamma \sigma}{r_{ik} - a\sigma} \right)
$$

- $r_{ij}, r_{ik}$ are pair distances
- $\theta$ is the angle $\angle jik$
- $\cos \theta_0$ defines the preferred bond angle (e.g., tetrahedral for silicon)

The **three-body term** contains an angular component of the form:

$$
\left( \cos \theta_{jik} + \frac{1}{3} \right)^2
$$

This function is minimized when $\cos \theta = -\frac{1}{3}$, corresponding to the **tetrahedral angle** $\theta \approx 109.47^\circ$. As a result, the potential energetically favors tetrahedral coordination and stabilizes the **diamond cubic lattice** found in silicon.

The original parameters determined by Stillinger and Weber are:

| Parameter | Value        | Unit |
|-----------|--------------|------|
| $\sigma$     | 2.0951       | $\mathrm{\AA}$    |
| $\varepsilon$   | 2.1682       | eV   |
| A         | 7.049556277  | —    |
| B         | 0.6022245584 | —    |
| p         | 4            | —    |
| q         | 0            | —    |
| a         | 1.8          | —    |
| $\lambda$ | 21.0          | — |
| $\mu$ | 1.2          | — |


### Only two-body term:
Assuming ideal bond angles, **we restrict our model to the two-body interaction term**.  

Let's plot it and see what it looks like.

In [ ]:
# SW potential parameters
A = 7.049556277
B = 0.6022245584
p = 4
q = 0
a = 1.8 
sigma = 2.0951  # Ångstroms
epsilon = 2.1682  # eV

# Define v2(r)
def v2_real_func(r):
    r_red = r / sigma
    if r >= a * sigma:
        return 0.0
    return epsilon * A * (B * r_red**(-p) - r_red**(-q)) * np.exp(1 / (r_red - a))

# Compute data
r_vals_real = np.linspace(1.8, 4.0, 500)
v2_vals = [v2_real_func(r) for r in r_vals_real]

# Important distances
r_NN = 2.35
r_cut = a * sigma

# Plot
plt.figure(figsize=(5, 4))
plt.plot(r_vals_real, v2_vals, color='midnightblue')

# Vertical lines
plt.axvline(x=r_NN, color='gray', linestyle='--')
plt.axvline(x=r_cut, color='red', linestyle=':')

# Annotations
plt.annotate(r'$r_{NN}$', xy=(r_NN, 0), xytext=(r_NN + 0.1, 0.2),
             arrowprops=dict(arrowstyle='->'), fontsize=12)

plt.annotate(fr'$r_{{\mathrm{{cut}}}}$', 
             xy=(r_cut, 0), xytext=(r_cut - 0.2, -0.4),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')

# Axes
plt.xlabel(r'$r\,\mathrm{[\AA]}$')
plt.ylabel(r'$v_2(r)\,\mathrm{[eV]}$')
plt.grid(alpha=0.5)
plt.tight_layout()
plt.show()

---

# b) Parameters and Force
- <span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> What is the relationship between the parameter $\varepsilon$ and the depth of the potential well? Illustrate qualitatively how the shape of the potential changes as 
$\varepsilon$ varies.  
  -  Hint: You can change `epsilon` in the code snippet and see how the plot changes.

- <span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> Given the shape of the two-body term in the Stillinger-Weber potential, sketch the corresponding force $F(r)$ as a function of distance $r$. What is the relationship between force and potential?

- <span style="color:blue; font-variant:small-caps;">(PYTHON)</span> Execute the code that calculates the force numerically using a finite-difference approximation of the potential and plots it.

- <span style="color:blue; font-variant:small-caps;">(PYTHON)</span> By setting the interatomic force to zero, we determine the distance $r_{ij}$ at which the potential reaches its minimum. This distance corresponds to the equilibrium bond length between first neighbors. Run the code and compare your result to the experimental value of $2.35,\text{\AA}$.

- <span style="color:blue; font-variant:small-caps;">(PYTHON)</span> Next, evaluate `v2_real_func` at `r_min` to obtain the energy at the minimum: do you recognize the value?.


In [ ]:
# Define numerical derivative: force = -dv2/dr
def force_real_func(r, h=1e-6):
    return -(v2_real_func(r + h) - v2_real_func(r - h)) / (2 * h)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(5, 6))

r_vals_real = np.linspace(1.8, 4, 500)
v2_vals = [v2_real_func(r) for r in r_vals_real]
force_vals = [force_real_func(r) for r in r_vals_real]


ax1.plot(r_vals_real, v2_vals, color='midnightblue')
ax1.axvline(x=2.35, color='gray', linestyle='--')
ax1.axvline(x=a * sigma, color='red', linestyle=':')
# Annotations
ax1.annotate(r'$r_{NN}$', xy=(r_NN, 0), xytext=(r_NN + 0.1, 0.2),
             arrowprops=dict(arrowstyle='->'), fontsize=12)

ax1.annotate(fr'$r_{{\mathrm{{cut}}}}$', 
             xy=(r_cut, 0), xytext=(r_cut - 0.2, -0.4),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')
ax1.set_ylabel(r'$v_2(r)\ \mathrm{[eV]}$')

ax1.grid(alpha=0.5)


# Plot force with LaTeX labels
ax2.plot(r_vals_real, force_vals, color='orange')
ax2.axvline(x=2.35, color='gray', linestyle='--')
ax2.axvline(x=a * sigma, color='red', linestyle=':')
ax2.set_xlabel(r'$r\ \mathrm{[\AA]}$')
ax2.set_ylabel(r'$f_{ij}(r) = -\dfrac{dv_2}{dr}\ \mathrm{[eV/\AA]}$')
ax2.grid(alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Use minimize_scalar on |force| to find the root of the force
force_root_result = minimize_scalar(lambda r: abs(force_real_func(r)),
                                    bounds=(2.0, a * sigma - 1e-4),
                                    method='bounded')

r_NN = force_root_result.x if force_root_result.success else None

print(f'{r_NN:.4f}')

In [ ]:
E_min = v2_real_func(r_min)

print(f'{E_min:.4f}')

<div class="alert alert-block alert-light">

## Solution

### Force calculation
We compute the gradient of the potential to find the force between atoms $i$ and $j$:

$$
\begin{align*}
- \nabla_{\mathbf{r}_i} v_2(r_{ij}) 
&= - \frac{\partial}{\partial \mathbf{r}_i} \left\{ \varepsilon A \left( B r_{ij}^{-p} - r_{ij}^{-q} \right) \exp \left[ \left( r_{ij} - a \right)^{-1} \right] \right\} \\
&= - \varepsilon A \frac{\mathbf{r}_{ij}}{r_{ij}} \left( \left[ \frac{\partial}{\partial r} \left( B r^{-p} - r^{-q} \right) \right] \bigg|_{r_{ij}} \exp \left[ \left( r_{ij} - a \right)^{-1} \right] \right. \\
&\quad + \left. \left( B r_{ij}^{-p} - r_{ij}^{-q} \right) \left\{ \frac{\partial}{\partial r} \exp \left[ (r - a)^{-1} \right] \right\} \bigg|_{r_{ij}} \right) \\
&= - \varepsilon A \frac{\mathbf{r}_{ij}}{r_{ij}} \left\{ \left( -p B r_{ij}^{-p-1} + q r_{ij}^{-q-1} \right) \exp \left[ (r_{ij} - a)^{-1} \right] \right. \\
&\quad - \left. \left( B r_{ij}^{-p} - r_{ij}^{-q} \right) \exp \left[ (r_{ij} - a)^{-1} \right] (r_{ij} - a)^{-2} \right\} \\
&= v_2 \frac{\mathbf{r}_{ij}}{r_{ij}} \left[ \frac{p B r_{ij}^{-p-1} - q r_{ij}^{-q-1}}{B r_{ij}^{-p} - r_{ij}^{-q}} + (r_{ij} - a)^{-2} \right] \\
&\equiv \mathbf{f}_{ij}
\end{align*}
$$

By solving ${f}_{ij} = 0$, we find:

$r_{\text{NN}} = 2.3517 \AA$.

This agrees well with the experimental value of $2.35,\text{\AA}$, as expected, since the potential was specifically fitted to reproduce such properties.

The depth of the potential well is $−2.1682\,\text{eV} = \varepsilon$.

---

# c) Lattice constant 

<span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> Use the nearest-neighbor distance to calculate the lattice constant, taking into account the crystal structure of silicon.

<div class="alert alert-block alert-light">

## Solution

Silicon crystallizes in a **diamond cubic** structure, which is a face-centered cubic (FCC) lattice with a 2-atom basis.

* In this structure, the **nearest-neighbor atoms** are along the body diagonals.
* The **NN distance** corresponds to:

  $$
  r_{\text{NN}} = \frac{\sqrt{3}}{4} a_0
  $$

  where $a$ is the **lattice constant**.


Solve for $a_0$:

$$
a_0 = \frac{4}{\sqrt{3}} \approx 5.431\ \mathrm{\AA}
$$

---

## d) Counting interactions

<span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> Based on your calculations and the given parameters, determine the number of neighboring atomic shells included within the Stillinger–Weber potential cutoff radius.

<div class="alert alert-block alert-light">

## Solution

In the **diamond cubic** structure, each atom has:

* **4 first-nearest neighbors**
* **12 second-nearest neighbors**
* **12 third-nearest neighbors**
* and so on...

But the **cutoff** of the SW potential is $r_{\text{cut}} = 1.8\,\sigma = 3.7712\ \text{Å}$

In the **diamond cubic structure**, there are **12** second neighbors are at a distance of:

  $$
  r_{\text{2nd}} = \frac{a}{\sqrt{2}} \approx \frac{5.43}{1.414} \approx 3.84\ \text{Å}
  $$

So:

* 1st NN: 4 neighbors at $r_1 = 2.3517$ Å $\implies$ included
* 2nd NN: 12 neighbors at $r_2 = \frac{a}{\sqrt{2}} \approx 3.84$ Å $\implies$ outside cutoff

---

# e) Compute energy per atom

- <span style="color:green; font-variant:small-caps;">(PEN & PAPER)</span> Compute the energy per atom by considering only first-neighbor interactions and neglecting any angular (directional bonding) contributions.

- <span style="color:blue; font-variant:small-caps;">(PYTHON)</span> From this, we can compute the equation of state $E(V)$, which describes how the total energy per atom varies with the atomic volume. In practice, this involves sweeping over different values of the lattice parameter, since the volume of the unit cell (and hence the volume per atom) depends directly on the lattice parameter. For each value of the lattice parameter $a_0$, we calculate the corresponding energy per atom considering that for the diamond cubic structure, the unit cell is cubic and contains 8 atoms.

In [ ]:
def energy_per_atom_NN(a0):
    r_NN = (np.sqrt(3) / 4) * a0
    v = v2_real_func(r_NN)
    z = 4                               
    return 0.5 * z * v                  

a0_vals = np.linspace(3.5, 8.0, 200)  
volumes = (a0_vals ** 3) / 8          
energies_NN = [energy_per_atom_NN(a0) for a0 in a0_vals]

# Plot E(V)
plt.figure(figsize=(5, 4))
plt.plot(volumes, energies_NN, color='forestgreen')
plt.xlabel(r'$V\,\mathrm{[\AA^3]}$')
plt.ylabel(r'$E(V)\,\mathrm{[eV]}$')
plt.grid(alpha=0.5)
plt.tight_layout()
plt.show()

<div class="alert alert-block alert-light">

## Solution

We'll compute the **energy per atom** by summing up the pairwise potential contributions from **all neighbors**, using the fact that:

$$
E_{\text{atom}} = \frac{1}{2} \sum_j v_2(r_{ij})
$$

The factor of ½ avoids double-counting since each interaction is shared between two atoms.

$$
E_{\text{atom}} = \frac{1}{2} \sum_{j=1}^{4} v_2(r_{ij}) = 2 \cdot v_2(r_{\text{NN}}) = 2 \varepsilon,
$$

The **two-body energy per atom** in the diamond structure of silicon, using the Stillinger–Weber potential and including only first-neighbor interactions, is then:

$$
\boxed{E_{\text{atom}}^{(2)} \approx -4.34\ \mathrm{eV}}
$$


> **CHECK: shouldn't the potential have been fitted to reproduce exactly the cohesive energy?** This is quite close to the known cohesive energy of silicon ($\approx 4.63\,\mathrm{eV/atom}$).

---

## f) Include second neighbors in the cutoff

<span style="color:blue; font-variant:small-caps;">(PYTHON)</span> Define a new cutoff value `a_modified` $> 1.8$ such that second-nearest neighbors are included.

- Plot both the pair potential and the equation of state with the new cutoff.

- Compare them to the original curves.

- What changes? Why? Briefly explain.

In [ ]:
# Cutoff values
a_original = 1.8
a_modified = 1.9  # new cutoff value

# Define v2 as a function of cutoff
def v2(r, a_cut):
    r_red = r / sigma
    if r >= a_cut * sigma:
        return 0.0
    return epsilon * A * (B * r_red**(-p) - r_red**(-q)) * np.exp(1 / (r_red - a_cut))

# Distance values
r_vals = np.linspace(1.8, a_modified * sigma - 1e-4, 500)
v2_vals_orig = [v2(r, a_original) for r in r_vals]
v2_vals_mod = [v2(r, a_modified) for r in r_vals]

# Important distances
r_NN = 2.3517
r_2NN = 3.84
r_cut_orig = a_original * sigma
r_cut_mod = a_modified * sigma

# Plot
plt.figure(figsize=(5, 4))
plt.plot(r_vals, v2_vals_orig, color='midnightblue', label=r'$v_2(r)$, original')
plt.plot(r_vals, v2_vals_mod, color='cadetblue', lw=1.5, label=r'$v_2(r)$, modified')

# Vertical lines
plt.axvline(x=r_NN, color='gray', linestyle=':')
plt.axvline(x=r_2NN, color='gray', linestyle=':')
plt.axvline(x=r_cut_orig, color='midnightblue', linestyle='--')
plt.axvline(x=r_cut_mod, color='cadetblue', linestyle='--')

# Annotations
plt.annotate(r'$r_{NN}$', xy=(r_NN, -0.5), xytext=(r_NN + 0.1, -1),
             arrowprops=dict(arrowstyle='->'), fontsize=12)
plt.annotate(r'$r_{2NN}$', xy=(r_2NN, -0.5), xytext=(r_2NN - 0.5, -1),
             arrowprops=dict(arrowstyle='->'), fontsize=12)
plt.annotate(fr'$r^{{\mathrm{{orig}}}}_{{\mathrm{{cut}}}}$', 
             xy=(r_cut_orig, -1.5), xytext=(r_cut_orig - 0.4, -2),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')
plt.annotate(fr'$r^{{\mathrm{{mod}}}}_{{\mathrm{{cut}}}}$', 
             xy=(r_cut_mod, -2), xytext=(r_cut_mod - 0.6, -2.5),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='tab:red')

# Axes
plt.xlabel(r'$r\ \mathrm{[\AA]}$')
plt.ylabel(r'$v_2(r)\ \mathrm{[eV]}$')
plt.grid(alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
a_modified = 1.9


# Energy per atom as function of lattice parameter a0
def energy_per_atom_2NN(a0):
    r_NN = (np.sqrt(3) / 4) * a0
    r_2NN = a0 / np.sqrt(2)

    v_NN = v2(r_NN, a_modified)
    v_2NN = v2(r_2NN, a_modified)

    z1 = 4   # number of 1st neighbors in diamond cubic
    z2 = 12  # number of 2nd neighbors

    return 0.5 * (z1 * v_NN + z2 * v_2NN)

# Volume per atom
def volume_per_atom(a0):
    return a0**3 / 8

# Sweep over a0 values
a0_vals = np.linspace(3.5, 8.0, 200)
volumes = [volume_per_atom(a0) for a0 in a0_vals]
energies_2NN = [energy_per_atom_2NN(a0) for a0 in a0_vals]

# Plot E(V)
plt.figure(figsize=(5, 4))

plt.plot(volumes, energies_NN, color='forestgreen', label='EOS, original')
plt.plot(volumes, energies_2NN, color='tomato', label='EOS, modified')
plt.xlabel(r'$V\,\mathrm{[\AA^3]}$')
plt.ylabel(r'$E(V)\,\mathrm{[eV]}$')

plt.grid(alpha=0.5)
plt.tight_layout()
plt.legend()
plt.show()

<div class="alert alert-block alert-light">

## Solution

The Stillinger–Weber pair potential includes a distance-based cutoff via an exponential function:

$$
v_2(r) = \varepsilon A \left( B r^{-p} - r^{-q} \right) \exp\left[\frac{1}{r/\sigma - a} \right]
\quad \text{for } r/\sigma < a
$$

### Effect on the Shape of the Pair Potential

When the cutoff parameter $a$ is increased without refitting the potential, the exponential cutoff decays more slowly. This extends the interaction range and allows weak mid-range attractive interactions from second-nearest neighbors to contribute to the potential. 

Although these interactions are individually small, they are more numerous (12 second neighbors vs. 4 first neighbors per atom), and their cumulative effect deepens the potential well and shifts its minimum slightly outward. These changes were **not accounted for during the original parameterization**, which leads to systematic errors such as overbinding and an exaggerated equilibrium bond length.

### Effect on the Shape of the Equation of State (EOS)

The consequences of increasing the cutoff extend beyond the pair potential itself. The Stillinger–Weber model for silicon is designed to include only first-neighbor interactions in the diamond structure. Extending the cutoff to include second-nearest neighbors without reparametrizing fundamentally alters the model’s assumptions.

At small lattice constants (i.e., under compression), second-nearest neighbors are brought closer and begin to behave like additional first neighbors. However, since their number is much larger (12 vs. 4), the model begins to treat the local structure more like a close-packed configuration (e.g., fcc) rather than tetrahedral. Yet the potential still applies interaction strengths calibrated for a tetrahedral environment. This mismatch leads to **unphysical behavior** in the energy–volume curve.

The underlying issue is that **the EOS only makes sense when the local bonding character remains consistent** as the volume changes. If increasing the cutoff causes a structural shift, implicitly blending different bonding geometries, then the resulting energy curve is no longer meaningful as a description of a single phase. It effectively interpolates between distinct phases without modeling the transition explicitly.

> A model is only valid within the assumptions it was built on. If you change those assumptions, you may still get plots and numbers, but they might no longer reflect anything physically meaningful.